In [ ]:
import numpy as np
from kilt.fit.mem_1d import scan_irf_and_eta_1d
from kilt.core.kernels import gaussian_irf, exp_kernel, make_irf_matrix_conv
from kilt.types import ILT1DConfig
from kilt.viz import plot_1d_fit

In [ ]:
"""Runnable example to run 1D MEM ILT on synthetic data."""

import numpy as np
from kilt.fit.mem_1d import scan_irf_and_eta_1d
from kilt.core.kernels import gaussian_irf, exp_kernel, make_irf_matrix_conv
from kilt.types import ILT1DConfig
from kilt.viz import plot_1d_fit

# time axis and synthetic two-exponential signal
time = np.linspace(0, 10, 300)
tau_true = np.array([1.5, 3.5])
amp_true = np.array([1.0, 0.4])
ExpData = amp_true[0]*np.exp(-time/tau_true[0]) + amp_true[1]*np.exp(-time/tau_true[1])
ExpData += 0.02 * np.random.randn(len(time)) * np.max(ExpData)
# tau grid
tau = np.linspace(0.1, 6, 80)
irf = gaussian_irf(time, center=0.5, width=0.2)
cfg = ILT1DConfig(time=time, tau=tau, irf=irf,
                    eta_grid=np.logspace(-8, -4, 4),
                    irf_shift_list=np.arange(0, 4), conv_pad=8,
                    m_prior=np.ones_like(tau)/len(tau),
                    y0=ExpData.min(),
                    verbose=True)
                    
result = scan_irf_and_eta_1d(ExpData, time, tau, irf, cfg)
print("Best loss:", result.best_loss, "Best eta:", result.best_eta, "IRF shift:", result.best_irf_shift)
fig = plot_1d_fit(time, ExpData, result.outputs['fit_curve'], result.outputs['A'], tau)
fig.show()

